<a href="https://colab.research.google.com/github/Kriszti8828/6het/blob/mergel/beadando_maxmin_ipynb_m%C3%A1solata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Package-ek importálása

In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import scipy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Adatbázisok importálása, tisztítás

In [ ]:
#Adatbázisok importálása
df_dbe=pd.read_csv('/content/drive/My Drive/numpu4/beadando_etfs/DBE.csv') #DBE
df_pbj=pd.read_csv('/content/drive/My Drive/numpu4/beadando_etfs/PBJ.csv') #PBJ
df_xlb=pd.read_csv('/content/drive/My Drive/numpu4/beadando_etfs/XLB.csv') #XLB
df_bil=pd.read_csv('/content/drive/My Drive/numpu4/beadando_etfs/BIL.csv') #BIl
df_qqq=pd.read_csv('/content/drive/My Drive/numpu4/beadando_etfs/QQQ.csv') #QQQ


In [ ]:
#YC importálása
df_yc=pd.read_csv('/content/drive/My Drive/numpu4/beadando_etfs/DGS3MO.csv')
#df_yc.head()
#az értelmezhetetlen karakter eliminációja, számmá alakítás
df_yc['DGS3MO']=df_yc['DGS3MO'].replace('.',10000).astype(float)
df_yc['DGS3MO']=df_yc['DGS3MO'].astype(float)

#És a nagy szám helyett a két szomszédos elem átlagának a beírása
for i in range(len(df_yc)):
  if df_yc['DGS3MO'][i]==10000:
    df_yc['DGS3MO'][i]=(df_yc['DGS3MO'][i-1]+df_yc['DGS3MO'][i+1])/2
  if df_yc['DGS3MO'][i]==0:
    df_yc['DGS3MO'][i]==df_yc['DGS3MO'][i-1]




In [ ]:
#A megfelelő hozamok közö dataframebe illesztése
#Az üres df. létrehozása
df_hozam=pd.DataFrame(columns={'Date','dbe','pbj','xlb','bil','qqq','yc'})
#Feltöltés a dátummal
df_hozam['Date']=df_qqq['Date'].copy()

In [ ]:
#Feltöltés az ETF-ek adj. close adatával
df_hozam['dbe']=df_dbe['Adj Close'].copy()
df_hozam['pbj']=df_pbj['Adj Close'].copy()
df_hozam['xlb']=df_xlb['Adj Close'].copy()
df_hozam['bil']=df_bil['Adj Close'].copy()
df_hozam['qqq']=df_qqq['Adj Close'].copy()

#és a hozamgörbével
df_hozam['yc']=df_yc['DGS3MO'].copy()


In [ ]:
#A dátum szerint indexelünk
df_hozam=df_hozam.set_index('Date')
df_hozam.index = pd.to_datetime(df_hozam.index)

In [ ]:
#Ellenőrzés
df_hozam.head(20)

In [ ]:
#a hozam kiszámítása
df_ret=pd.DataFrame(columns={'dbe','pbj','xlb','bil','qqq','yc'})
df_ret['dbe']=df_hozam['dbe']/df_hozam['dbe'].shift(1)-1
df_ret['pbj']=df_hozam['pbj']/df_hozam['pbj'].shift(1)-1
df_ret['xlb']=df_hozam['xlb']/df_hozam['xlb'].shift(1)-1
df_ret['bil']=df_hozam['bil']/df_hozam['bil'].shift(1)-1
df_ret['qqq']=df_hozam['qqq']/df_hozam['qqq'].shift(1)-1
df_ret['yc']=(df_hozam['yc']+1)/(df_hozam['yc']+1).shift(1)-1


In [ ]:
#A dátum szerint indexelünk
df_ret.index = pd.to_datetime(df_ret.index)

In [ ]:
#Ellenőrzés
df_ret.head(10)

In [ ]:
#A hozamgörbe manipulálása
from math import nan
#df_ret['yc'].plot()
for i in range(len(df_ret['yc'])):
  if df_ret['yc'][i]==np.NaN:
    df_ret['yc'][i]=0
  if df_ret['yc'][i]<-100:
    df_ret['yc'][i]=0
  if df_ret['yc'][i]>100:
    df_ret['yc'][i]=0

In [ ]:
#Ellenőző plot az etf-ekhez
df_ret['xlb'].plot(linewidth=0.3)

In [ ]:
print(df_ret['yc'][0:252*5].mean()*252)

In [ ]:
#A végtelen és a 0 adatok kezelése
df_ret=df_ret.replace(np.inf,np.nan).astype(float)

In [ ]:
#Az yc nélküli hozamok, adatbázis
df_hozam2=df_hozam.copy()
#df_hozam2.head()
df_hozam2=df_hozam2.drop(['yc'], axis=1)
df_hozam_gsh=df_hozam2.copy() #Adatbázis a globális Sharpe-rátához
df_hozam_lsh=df_hozam2.copy() #Adatbázis a csúszóablakos Sharpe-rátához

###Sharpe ráta, globális

#####A 10-11. órán felépített kódokat használva:

In [ ]:
def minden(price_df,w):
  return_asset = price_df/price_df.shift(1)-1
  mean_asset = return_asset.mean() * 252       #évestett átlag
  std_asset = return_asset.std() * np.sqrt(252)  #évesített szórás
  cov_asset = return_asset.cov() * 252
  corr_asset = return_asset.corr()

#Itt jön be a calc nasset mean és cov

  #N eszköz átlagos hozama, a megfelelő súlyokkal
  calc_nasset_mean=np.sum(w*mean_asset)    #2 vektor elemenkénti szorzata --> összeadjuk

  #n eszköz kovarianciája a megfelelő súlyokkal
  calc_nasset_cov=np.sqrt(np.dot(np.dot(w,cov_asset), w.transpose()))   #np.dot  mátrix szorzat

  #A return és a szórás meghatározása
  ret = calc_nasset_mean
  std = calc_nasset_cov
  #Kockázatmentes hozamgörbe  
  
 
  return ret, std


In [ ]:
ww=np.array([0,0,0,1,0])
#A fenti függvény behívása
rreturn, standev =minden(df_hozam_gsh, ww)


In [ ]:
#Teszt, vektorok
print(rreturn)
print(standev)
print(rreturn/standev)

In [ ]:
#Kockázatmentes hozam
kockment=(df_ret['yc']).mean()*252

#####Optimalizálás

In [ ]:
#A célfüggvény
def elsoopt(ww):
  a1,a2=minden(df_hozam_gsh,ww)
  Sh_rate=(a1-kockment)/a2
  return Sh_rate

In [ ]:
fun1 = lambda x: 100-elsoopt(x) #ide proba(x) is jöhet tesztelni

In [ ]:
bounds=((0,None),(0,None),(0,None),(0,None),(0,None))
cons=({'type': 'eq', 'fun': lambda ww: np.sum(ww)-1})

In [ ]:
#Az optimalizálás (~3 min volt nekem)
megoldas1_1=scipy.optimize.minimize(fun1, (0.5,0.2,0.1,0.0,0), method='SLSQP',
                        bounds=bounds,
                        constraints=cons)

In [ ]:
#Az optimalizálás eredménye
print(megoldas1_1)

In [ ]:
#És az eredménnyel számolt hozam
rreturn, standev=minden(df_hozam_gsh,megoldas1_1['x'])
#A kapott Sharpe-ráta
print((rreturn-0.01)/standev)
#Ellenőrzés: az öszeg valóban 1
print(sum(megoldas1_1['x']))
#A kapott hozam
print(elsoopt(megoldas1_1['x']))

##Csúszóablakos Shapre ráta


In [ ]:
def masodikopt(start,stop): #A fenti függvény általánosítása az időablakokhoz
#  start=5
#  stop=25
  megoldas1_1=[]
  

  
  for i in range(start,stop):
    
    kockment=(df_ret['yc'][(start+i):(start+i+5*252)]).mean()*252 #évesített, 5 éves ablakot néz
    def celfv(ww):
        a1,a2=minden(df_hozam_gsh[i:(i+5*252)],ww)
        Sh_rate=(a1-kockment)/a2      
        return Sh_rate

    funa = lambda x: 100-celfv(x) #max: Szám-x akkor min., ha x max.
    bounds=((0,None),(0,None),(0,None),(0,None),(0,None))
    cons=({'type': 'eq', 'fun': lambda ww: np.sum(ww)-1})

    megoldas1_a=scipy.optimize.minimize(funa, (0.5,0.2,0.1,0.0,0), method='SLSQP',
                          bounds=bounds,
                          constraints=cons)
    megoldas1_1.append(megoldas1_a['x'])
  #for i in len(megolda)
  return megoldas1_1

In [ ]:
tesztsok=masodikopt(1,len(df_hozam2['bil']-5*252-1)) #A LE NE FUTTASD! #A futásidő ~8 perc
#print(tesztsok)
print(df_hozam_gsh.head(4))

In [ ]:
megoldas1_2=pd.DataFrame(columns={'x1','x2','x3','x4','x5'}) #A kapott megoldás táblázattá alakítása
megoldas1_2['x1']=np.zeros(len(tesztsok))
megoldas1_2['x2']=np.zeros(len(tesztsok))
megoldas1_2['x3']=np.zeros(len(tesztsok))
megoldas1_2['x4']=np.zeros(len(tesztsok))
megoldas1_2['x5']=np.zeros(len(tesztsok))
for i in range(len(tesztsok)):
  megoldas1_2['x1'][i]=tesztsok[i][0]
  megoldas1_2['x2'][i]=tesztsok[i][1]
  megoldas1_2['x3'][i]=tesztsok[i][2]
  megoldas1_2['x4'][i]=tesztsok[i][3]
  megoldas1_2['x5'][i]=tesztsok[i][4]

In [ ]:
#Az eredmények csv-re mentése:
with open('megoldas1_2.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(megoldas1_2)
print(megoldas1_2)

In [ ]:
megoldas1_2=pd.read_csv('/content/drive/My Drive/numpu4/beadando_etfs/megoldas1_2.csv') #Hogy ne kelljen mindig újrafuttatni, itt a korábban megkapott optimumot hívjuk be
print(megoldas1_2)


In [ ]:
#x=df_bil['Date']
ay = megoldas1_2['x1'] #pbj
by = megoldas1_2['x2'] #dbe
cy = megoldas1_2['x3'] #xlb
dy = megoldas1_2['x4'] #qqq
ey = megoldas1_2['x5'] #bil
y = np.vstack([ay, by, cy,dy,ey])
sulyok=np.array([ay, by, cy,dy,ey])
# plot
fig, ax = plt.subplots(figsize=(10,7))
x=df_bil['Date'][0:len(ay)]
plt.xlim=(x)
ax.stackplot(megoldas1_2.index, y)
plt.legend(['xlb','dbe','bil','qqq','pbj'])
plt.suptitle('Sharpe-ráta gördülő súlyok')
plt.ylabel('%')
plt.savefig("gordulosharpe.png") 
plt.show()

In [ ]:
###################Returnök kiszámítása

In [ ]:
#Returnök visszaadása: üres adatbázis létrehozása
gordreturn=pd.DataFrame(columns={'return'})
gordreturn['return']=np.zeros(len(sulyok[0]))

In [ ]:
#Ellenőrzés
df_ret.head(3)

In [ ]:
#A súlyfüggvények
print(sulyok)

In [ ]:
#A súlyfüggvények np. array-é alakítása, mert így tud lefutni a 'minden' kód
df_suly1_2=megoldas1_2[['x1','x2','x3','x4','x5']]
sulyarray=[]

for i in range(len(df_suly1_2['x1'])):
  sulyarray.append([])
  for j in range(5):
    sulyarray[i].append(df_suly1_2[df_suly1_2.columns[j]][i])
  sulyarray[i]=np.array(sulyarray[i])

In [ ]:
#Ellenőrzés: valóban np. array
print(type(sulyarray[1]))

In [ ]:
#A returnök kiszámolása a csúszóablakos módszernél
gordreturn['return']=np.zeros(len(megoldas1_2['x1']))
for i in range(len(megoldas1_2['x1'])):  
  gordreturn['return'][i]=minden(df_hozam_gsh[i:(i+5*252)],sulyarray[i])[0]
  #df_ret[df_ret.columns[1]]*megoldas1_2['x1'][i] #pbj #az index 1-nél indul, mert a 0. az yc
  

In [ ]:
#Dátum hozzáadása
gordreturn['Date']=df_bil['Date'][0:len(gordreturn['return'])]
print(gordreturn.head(4))
gordreturn=gordreturn.set_index('Date')
gordreturn.index = pd.to_datetime(gordreturn.index)

In [ ]:
#A csúszóablakos optimális returnök ábrázolása
gordreturn['return'][1:3600].plot()
plt.figure(figsize=(12, 9))
plt.suptitle('Portfólió alakulása')
plt.ylabel('Hozam')
plt.xlabel('Dátum')
plt.savefig("returngordsharpe.png") 
plt.show()

In [ ]:
#minden(df_hozam_gsh[1:(1+5*252)],df_suly1_2[1:(1+1)])
minden(df_hozam_gsh,np.array([1,0,0,0,0]))

##minmax

#####Adatbázis kialakítása

######A fentiektől eltérően itt naponta kell hozamot vizsgálni (a min. max. drawdownhoz), 

In [ ]:
#Súlyok
w=[0,1,0,0,0]

In [ ]:
print(df_hozam2)

In [ ]:
#Létrehozunk egy eredmény oszlopot a súlyokkal számolt returnhöz
df_hozam2['eredmeny']=0
print(df_hozam2['eredmeny'])

In [ ]:
#Feltöltjük
df_hozam2['eredmeny']=df_hozam2['eredmeny']+df_ret['dbe']*w[0]
df_hozam2['eredmeny']=df_hozam2['eredmeny']+df_ret['qqq']*w[1]
df_hozam2['eredmeny']=df_hozam2['eredmeny']+df_ret['pbj']*w[2]
df_hozam2['eredmeny']=df_hozam2['eredmeny']+df_ret['xlb']*w[3]
df_hozam2['eredmeny']=df_hozam2['eredmeny']+df_ret['bil']*w[4]


In [ ]:
#2. lehetőség: for ciklust írunk rá
df_hozam3=pd.DataFrame(columns={'eredmeny'})
df_hozam3['eredmeny']=df_hozam2['eredmeny'].copy()
df_hozam3['eredmeny']=0
print(df_hozam3.head(5))

In [ ]:
#Feltöltjük a skalárszorzatokkal
for i in range(len(df_hozam2.columns)-2):
  df_hozam3['eredmeny']=df_hozam3['eredmeny']+df_ret[df_hozam2.columns[i]]*w[i]

print(df_hozam3['eredmeny'])

In [ ]:
#oszlop meghatározása
#df_hozam2.columns[1]
#df_hozam2[df_hozam2.columns[1]]


In [ ]:
#Teszt
print(df_hozam2['eredmeny'])

In [ ]:
#A portfólió hozamának a kiszámítása
df_hozam2['return']=np.zeros
df_hozam2['return'][0]=1

df_hozam2['return']=df_hozam2['eredmeny']
df_hozam2['return'][0]=1
df_hozam2['return'][1]=df_hozam2['return'][0]*(1+df_hozam2['eredmeny'][1])



for i in range(len(df_hozam2['eredmeny'])-1):
  df_hozam2['return'][i+1]=(df_hozam2['eredmeny'][i+1]+1)*df_hozam2['return'][i]

In [ ]:
#df_hozam2['return']=df_hozam2['eredmeny']
#for i in range(len(df_hozam2['eredmeny'])-1):
#  df_hozam2['return'][i]=df_hozam2['eredmeny'][i+1]*df_hozam2['eredmeny'][i]

In [ ]:
#Ellenőrzések
#print(df_hozam2['return'],df_hozam2['eredmeny'])
#df_hozam2[['eredmeny','return']].plot()
df_hozam2[['return','eredmeny']].plot(title="A portfólió hozama, és az egyes időszakok eredménye")
plt.show()


In [ ]:
#A legiksebb és legnagyobb érték kiszámolása
#kicsi=min(df_hozam2['return'])
#nagy=max(df_hozam2['return'])
#print(nagy-kicsi)

In [ ]:
def hozamdf(df1,df2, ww):

  adatsor=df1.copy() #így annyi sorból áll, mint az eredeti df
  adatsor['eredmeny']=0
  for i in range(len(df1.columns)-2):
    adatsor['eredmeny']=adatsor['eredmeny']+df2[df1.columns[i]]*ww[i] #a súlyokkal súlyozunk, hogy megfelelő legyen a hozam


  adatsor['return']=np.zeros
  adatsor['return'][0]=1
  adatsor['return'][1]=adatsor['return'][0]*(1+adatsor['eredmeny'][1]) #kiszámítjuk a hozamot

  for i in range(len(adatsor['eredmeny'])-1):
    adatsor['return'][i+1]=(adatsor['eredmeny'][i+1]+1)*adatsor['return'][i]

  return adatsor['return']

In [ ]:
#Ellenőrzés
qq=hozamdf(df_hozam2,df_ret,w)
#print(qq.head(-5))
fig=qq.plot(title="A portfólió hozamának alakulása az adott w súlyokkal",
            )
#fig.legend="return"

plt.show(fig)

In [ ]:
def maxmin(ww):
  qq=hozamdf(df_hozam2,df_ret,ww)
  kicsi=min(qq)
  a=0 
  for i in qq:
    a=a+1
    if i == kicsi: 
      break
  nagy=0
  for j in range(0,a):
    if qq[j]>nagy:
      nagy=qq[j]
  return (nagy-kicsi)/nagy


In [ ]:
drawd=maxmin(w)
print(drawd)

####Optimalizálás

In [ ]:
#Szükséges package-ek importálása
import seaborn as sns

import scipy

In [ ]:
#Egy egyszerűen ellenőrizhető példafüggvény
def proba(ww):
  return ww[0]/ww[1]+ww[2]+ww[3]+ww[4]

In [ ]:
#Az optimalizálandó függvény
fun3 = lambda x: maxmin(x) #ide proba(x) is jöhet tesztelni

In [ ]:
#Határok és korlátok
bounds=((0,None),(0,None),(0,None),(0,None),(0,None))
cons=({'type': 'eq', 'fun': lambda ww: np.sum(ww)-1})

In [ ]:
#Az optimalizálás (~3 min volt nekem)
megoldas3=scipy.optimize.minimize(fun3, (0.1,0.1,0.1,0.1,0.6), method='SLSQP',
                        bounds=bounds,
                        constraints=cons)

In [ ]:
#Az eredmény
print(megoldas3)

In [ ]:
#Ellenzőrzés
sum(megoldas3['x'])

In [ ]:
#És a portfólió alakulása az időben
drawd=maxmin(megoldas3['x'])
print(drawd)


In [ ]:
qq=hozamdf(df_hozam2,df_ret,megoldas3['x'])
#print(qq.head(-5))
#qq.plot()
print(min(qq))

####Optimalizálás más feltételekkel

In [ ]:
cons2=({'type': 'eq', 'fun': lambda ww: np.sum(ww)-1},
       {'type': 'ineq', 'fun': lambda ww: ww[1]-0.05})

In [ ]:
#Az optimalizálás (~3 min volt nekem)
megoldas3b=scipy.optimize.minimize(fun3, (0.1,0.1,0.1,0.1,0.6), method='SLSQP',
                        bounds=bounds,
                        constraints=cons2)

In [ ]:
print(megoldas3b)

####Minmax csúszóablakosan

In [ ]:
def negyedikopt(start,stop):
#  start=5
#  stop=25
  megoldas1_1=[]
  

  
  for k in range(start,stop):
    
    
    def maxmin(ww):
      qq=hozamdf(df_hozam2[(start+k):(start+5*252+k)],df_ret,ww)
      kicsi=min(qq)
      a=0 
      for i in qq:
        a=a+1
        if i == kicsi: 
          break
      nagy=0
      for j in range(0,a):
        if qq[j]>nagy:
          nagy=qq[j]
      return (nagy-kicsi)/nagy
    
    funa = lambda x: maxmin(x) #ide proba(x) is jöhet tesztelni
    bounds=((0,None),(0,None),(0,None),(0,None),(0,None))
    cons=({'type': 'eq', 'fun': lambda ww: np.sum(ww)-1})

    megoldas1_a=scipy.optimize.minimize(funa, (0.5,0.2,0.1,0.0,0), method='SLSQP',
                          bounds=bounds,
                          constraints=cons)
    megoldas1_1.append(megoldas1_a['x'])
  #for i in len(megolda)
  return megoldas1_1

In [ ]:
#teszt4=negyedikopt(200,250)

In [ ]:
#print(teszt4)



In [ ]:

megoldas3_2=pd.DataFrame(columns={'x1','x2','x3','x4','x5'})
megoldas3_2['x1']=np.zeros(len(tesztsok))
megoldas3_2['x2']=np.zeros(len(tesztsok))
megoldas3_2['x3']=np.zeros(len(tesztsok))
megoldas3_2['x4']=np.zeros(len(tesztsok))
megoldas3_2['x5']=np.zeros(len(tesztsok))
for i in range(len(tesztsok)):
  megoldas3_2['x1'][i]=tesztsok[i][0]
  megoldas3_2['x2'][i]=tesztsok[i][1]
  megoldas3_2['x3'][i]=tesztsok[i][2]
  megoldas3_2['x4'][i]=tesztsok[i][3]
  megoldas3_2['x5'][i]=tesztsok[i][4]

In [ ]:
#print(megoldas3_2.head(5))
megoldas3_2=pd.read_csv('/content/drive/My Drive/numpu4/beadando_etfs/megoldas3_2.csv')
print(type(megoldas3_2['x3']))

In [ ]:
#x=df_bil['Date']
ay = megoldas3_2['x1']
by = megoldas3_2['x2']
cy = megoldas3_2['x3']
dy = megoldas3_2['x4']
ey = megoldas3_2['x5']
y = np.vstack([ay,  by, cy,dy,ey]) #y = np.vstack([ay, by, cy,dy,ey])

# plot
fig, ax = plt.subplots(figsize=(10,7))
ax.stackplot(megoldas3_2.index, y)
plt.legend(['xlb','bil','qqq','pbj','qqq'])
plt.suptitle('Drawdown gördülő súlyok')
plt.ylabel('%')
plt.savefig("gordulodraw.png") 
plt.show()

In [ ]:
#tesztsok cs-ve mentése:
with open('megoldas3_2.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(megoldas3_2)
print(megoldas3_2)

In [ ]:
from pathlib import Path  
filepath = Path('/content/drive/My Drive/numpu4/beadando_etfs/megoldas3_2.csv')  #'/content/drive/My Drive/numpu4/beadando_etfs/DBE.csv'
filepath.parent.mkdir(parents=True, exist_ok=True)  
megoldas3_2.to_csv(filepath)

In [ ]:
############Returnök számolása

In [ ]:
df_suly3_2=megoldas3_2[['x1','x2','x3','x4','x5']]
sulyarray2=[]
for i in range(len(df_suly1_2['x1'])):
  sulyarray2.append([])
  for j in range(5):
    sulyarray2[i].append(df_suly1_2[df_suly1_2.columns[j]][i])
  sulyarray2[i]=np.array(sulyarray[i])

In [ ]:
#Ellenőrzés
print(sulyarray2[1])

In [ ]:
gordreturn_dd=pd.DataFrame(columns={'return'})
gordreturn_dd['return']=np.zeros(len(df_suly3_2['x1']))

In [ ]:
#A returnök kiszámolása a csúszóablakos módszernél
gordreturn_dd['return']=np.zeros(len(megoldas3_2['x1']))
for i in range(len(megoldas3_2['x1'])):  
  gordreturn_dd['return'][i]=minden(df_hozam_gsh[i:(i+5*252)],sulyarray[i])[0] #Itt is átlaghozamokat számolunk, hogy tudjuk ábrázolni
  #df_ret[df_ret.columns[1]]*megoldas1_2['x1'][i] #pbj #az index 1-nél indul, mert a 0. az yc
  

In [ ]:
#Dátum beállítása
gordreturn_dd['Date']=np.zeros
for i in range(len(gordreturn_dd)):
  gordreturn_dd['Date'][i]=df_bil['Date'][200+i]
print(gordreturn_dd.head(4))
#gordreturn_dd.index = pd.to_datetime(gordreturn_dd.index)

In [ ]:
gordreturn_dd=gordreturn_dd.set_index('Date')


In [ ]:
#'gordreturn_dd['Date']=df_bil['Date'][200:250]
print(gordreturn_dd)

In [ ]:
gordreturn_dd['return'].plot()


In [ ]:
#hozamdf(df_hozam2[(start+k):(start+5*252+k)],df_ret,ww)

##Példa a minimalizálásra

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

In [ ]:
fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2

bnds = ((0, None), (0, None))

cons = ({'type': 'ineq', 'fun': lambda x:  x[0] - 2 * x[1] + 2},
        {'type': 'ineq', 'fun': lambda x: -x[0] - 2 * x[1] + 6},
        {'type': 'ineq', 'fun': lambda x: -x[0] + 2 * x[1] + 2})

res = scipy.optimize.minimize(fun, (2, 0), method='SLSQP', bounds=bnds,
               constraints=cons)

In [ ]:
res

# Ábrák

In [ ]:
plt.figure(figsize=(12, 9))
plt.suptitle('amerikai kincstári értékpapírok piaci hozama 3 hónapos állandó lejáraton')
df_hozam['yc'].plot()
plt.ylabel('Hozam')
plt.xlabel('Dátum')
plt.savefig("yc_hozam.png") 
plt.show()

In [ ]:
df_hozam.plot( y=['dbe','pbj','bil','qqq', 'xlb'], figsize=(12,9))
plt.suptitle('Portfolió eszközök ára')
plt.ylabel('Korrigált záróár')
plt.xlabel('Dátum')
plt.savefig("assets-adjclose.png") 
plt.show()

In [ ]:
df_ret.head(50)
df_ret.plot( y=['dbe','pbj','bil','qqq', 'xlb'], figsize=(12,9))
plt.suptitle('Portfolió eszközök ára')
plt.ylabel('Megtérülés')
plt.xlabel('Dátum')
plt.savefig("assets-return.png") 
plt.show()


In [ ]:
plt.figure(figsize=(12, 9))
plt.suptitle('Kockázatmentes')
df_ret['yc'].plot()
plt.ylabel('Hozam')
plt.xlabel('Dátum')
plt.savefig("yc_return.png") 
plt.show()